In [1]:
import pandas as pd
import os
from tqdm import tqdm

# read the list of file names from the csv file
file_list = pd.read_csv('/home/tzahid/Desktop/pred_sensor_list.csv')['pred_sensor'].tolist()

# add '.csv' extension to file names in the list
file_list = [filename+'.csv' for filename in file_list]

# get a list of all csv files in the folder
path = '/home/tzahid/Desktop/dataset22/'
all_files = os.listdir(path)
csv_files = [filename for filename in all_files if filename.endswith('.csv') and filename in file_list]



print(f"{len(csv_files)} CSV files found matching with the list of file names.")


# get the range of index values
max_index = 0
for file in csv_files:
    try:
        data = pd.read_csv(os.path.join(path, file), usecols=[0])
        max_index = max(max_index, data['index_date'].max())
    except FileNotFoundError:
        print(f"File not found: {file}. Skipping to next file.")

index_range = range(1, max_index+1)

# create an empty dataframe with the desired index
df = pd.DataFrame(index=index_range)

# loop through the csv files and extract the 5th column and add it to the dataframe
for file in tqdm(csv_files, desc='Processing CSV files'):
    try:
        col_name = os.path.splitext(file)[0]
        col_name = col_name.replace(" ", "_")  # replace spaces with underscores in column names
        data = pd.read_csv(os.path.join(path, file), usecols=[0, 4], header=0, skiprows=[0], names=['index_date', col_name]) # modify usecols to define which columns to extract, here 0 and 4
        data = data.drop_duplicates(subset=['index_date'])  # drop any duplicate rows
        data = data.set_index('index_date')  # set the index to the 'index_date' column
        data = data.reindex(index_range, fill_value=0)  # add missing index rows and fill with 0
        df[col_name] = data[col_name]  # add the column to the main dataframe
    except FileNotFoundError:
        print(f"File not found: {file}. Skipping to next file.")

# save the concatenated dataframe to a new csv file
# name of the final file
df.to_csv(os.path.join(path, 'pred_sensor_values.csv'), index_label='index_date')

# print the range of index values
print(f"Index range: 1 - {max_index}")

# print final message
print(f"Concatenation complete. {len(df.columns)} files have been concatenated.")


26 CSV files found matching with the list of file names.


Processing CSV files: 100%|██████████| 26/26 [00:04<00:00,  5.82it/s]


Index range: 1 - 306846
Concatenation complete. 26 files have been concatenated.
